This notebook gives a simple combination of literal matching and Named Entity Recognition using BERT (base model from huggingface).

The training phase of the BERT model was done in another kernel: Pytorch BERT for Named Entity Recognition.

In [ ]:
MAX_SAMPLE = None # set a small number for experimentation, set None for production.
PREDICTION_SIZE = 100

# Install packages

In [ ]:
!pip install datasets --no-index --find-links=file:///kaggle/input/coleridge-packages/packages/datasets
!pip install ../input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
!pip install ../input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/coleridge-packages/transformers-4.5.0.dev0-py3-none-any.whl

# Import

In [ ]:
import os
import re
import json
import time
import datetime
import random
import glob
import importlib

import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

#new
from sklearn.model_selection import StratifiedKFold, train_test_split

random.seed(123)
np.random.seed(456)

# Load data

In [ ]:
train_path = '../input/coleridgeinitiative-show-us-the-data/train.csv'
ds = pd.read_csv(train_path)
ds = ds[:MAX_SAMPLE] #picks the number of samples from the trainingset

train, test = train_test_split(ds , test_size=0.20, random_state=42)
#append all texts after each other
paper_train_folder = '../input/coleridgeinitiative-show-us-the-data/train'
papers = {}
for paper_id in ds['Id'].unique():
    with open(f'{paper_train_folder}/{paper_id}.json', 'r') as f:
        paper = json.load(f)
        papers[paper_id] = paper

In [ ]:
sample_submission_path = '../input/coleridgeinitiative-show-us-the-data/sample_submission.csv'
sample_submission = pd.read_csv(sample_submission_path)

paper_test_folder = '../input/coleridgeinitiative-show-us-the-data/test'
for paper_id in sample_submission['Id']: #TODO make sure this uses the test data createdabove.
    with open(f'{paper_test_folder}/{paper_id}.json', 'r') as f: #Training with testdata? or maybe they don't use the test part of the papers dictionary
        paper = json.load(f)
        papers[paper_id] = paper

# Show statistic about the data

In [ ]:
print(f"Number of different datasets: {len(ds['dataset_label'].unique())}")
print(f"Size of the dataset: {len(ds['dataset_label'])}")

print("\nFrequency table")
print(ds['dataset_label'].value_counts())
print("\n")
ds['dataset_label'].value_counts().plot()

a = 4

# Literal matching

### Create a knowledge bank

Extra_data reads out the extra data sets that are retrieved from a notebook by Ken Miller (https://www.kaggle.com/mlconsult/score-57ish-with-additional-govt-datasets)

In [ ]:

all_labels = set()

for label_1, label_2, label_3 in train[['dataset_title', 'dataset_label', 'cleaned_label']].itertuples(index=False):
    all_labels.add(str(label_1).lower())
    all_labels.add(str(label_2).lower())
    all_labels.add(str(label_3).lower())
    
# Load in extra labels
extra_data = pd.read_csv('../input/bigger-govt-dataset-list/data_set_26897.csv')
    
# for x, row in extra_data.iterrows():
#     all_labels.add(str(row['title']).lower())
    
print(f'No. different labels: {len(all_labels)}')

### Matching on test data

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

def totally_clean_text(txt):
    txt = clean_text(txt)
    txt = re.sub(' +', ' ', txt)
    return txt

In [ ]:
literal_preds = []

for paper_id in test['Id'][:PREDICTION_SIZE]:
# for paper_id in sample_submission['Id']:
    paper = papers[paper_id]
    text_1 = '. '.join(section['text'] for section in paper).lower()
    text_2 = totally_clean_text(text_1)
    
    labels = set()
    for label in all_labels:
        if label in text_1 or label in text_2:
            labels.add(clean_text(label))
    
    literal_preds.append('|'.join(labels)) #separates the different datasets for the papers using |


In [ ]:
literal_preds[:5]

# Bert prediction

### Paths and Hyperparameters

In [ ]:
MAX_LENGTH = 64 # max no. words for each sentence.
OVERLAP = 20 # if a sentence exceeds MAX_LENGTH, we split it to multiple sentences with overlapping

PREDICT_BATCH = 64000 

PRETRAINED_PATH = '../input/split8020seed42/output'
TEST_INPUT_SAVE_PATH = './input_data'
TEST_NER_DATA_FILE = 'test_ner_input.json'
TRAIN_PATH = '../input/split8020seed42/train_ner.json'
VAL_PATH = '../input/split8020seed42/train_ner.json'

PREDICTION_SAVE_PATH = './pred'
PREDICTION_FILE = 'test_predictions.txt'

### Transform data to NER format

Group by publication, training labels should have the same form as expected output.

We split the full training data into a training and validation part, using train_test_split(). 

In [ ]:
train_full = train.groupby('Id').agg({
    'pub_title': 'first',
    'dataset_title': '|'.join,
    'dataset_label': '|'.join,
    'cleaned_label': '|'.join
}).reset_index()

train_path = '../input/coleridgeinitiative-show-us-the-data/train.csv'
paper_train_folder = '../input/coleridgeinitiative-show-us-the-data/train'

ds = pd.read_csv(train_path)
ds = ds[:MAX_SAMPLE]

train, test = train_test_split(ds , test_size=0.20, random_state=42) #TODO make sure this also happens for literal matching

print(f'No. grouped training rows: {len(train)}', f'No. grouped test rows: {len(test)}')

In [ ]:
def clean_training_text(txt):
    """
    similar to the default clean_text function but without lowercasing.
    """
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt)).strip()

def shorten_sentences(sentences):
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

We moved the ner format making to the BERT training notebook. This notebook returns train and test set.

In [ ]:
test_rows = [] # test data in NER format
paper_length = [] # store the number of sentences each paper has

for paper_id in test['Id'][:PREDICTION_SIZE]:
# for paper_id in sample_submission['Id']:
    # load paper
    paper = papers[paper_id]
    
    # extract sentences
    sentences = [clean_training_text(sentence) for section in paper 
                 for sentence in section['text'].split('.')
                ] #This is tricky for URLs, as this may lead to very short sentences containing 'google', thus below sentences shorter than 10 characters are filtered out.
    sentences = shorten_sentences(sentences) # make sentences short
    sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
    sentences = [sentence for sentence in sentences if any(word in sentence.lower() for word in ['data', 'study', 'dataset', 'et al.', 'https', 'http'])] #only adds sentence if it contains word like data and study ??
    
# TODO Check which words are often found in sentences with datasets.    
    
    # collect all sentences in json
    for sentence in sentences:
        sentence_words = sentence.split()
        dummy_tags = ['O']*len(sentence_words)
        test_rows.append({'tokens' : sentence_words, 'tags' : dummy_tags})
    
    # track which sentence belongs to which data point
    paper_length.append(len(sentences))
    
print(f'total number of sentences: {len(test_rows)}')

### Do predict and collect results

In [ ]:
os.environ["MODEL_PATH"] = f"{PRETRAINED_PATH}"
os.environ["TRAIN_FILE"] = f"{TRAIN_PATH}"
os.environ["VALIDATION_FILE"] = f"{VAL_PATH}"
os.environ["TEST_FILE"] = f"{TEST_INPUT_SAVE_PATH}/{TEST_NER_DATA_FILE}" #TODO Change the path to the new testset retrieved from the train notebook.
os.environ["OUTPUT_DIR"] = f"{PREDICTION_SAVE_PATH}"

In [ ]:
# copy my_seqeval.py to the working directory because the input directory is non-writable
!cp /kaggle/input/coleridge-packages/my_seqeval.py ./

# make necessart directories and files
os.makedirs(TEST_INPUT_SAVE_PATH, exist_ok=True)

In [ ]:
def bert_predict():
    !python ../input/kaggle-ner-utils/kaggle_run_ner.py \
    --model_name_or_path "$MODEL_PATH" \
    --train_file "$TRAIN_FILE" \
    --validation_file "$VALIDATION_FILE" \
    --test_file "$TEST_FILE" \
    --output_dir "$OUTPUT_DIR" \
    --report_to 'none' \
    --seed 123 \
    --do_predict

In [ ]:
bert_outputs = []

for batch_begin in range(0, len(test_rows), PREDICT_BATCH):
    # write data rows to input file
    with open(f'{TEST_INPUT_SAVE_PATH}/{TEST_NER_DATA_FILE}', 'w') as f:
        for row in test_rows[batch_begin:batch_begin+PREDICT_BATCH]:
            json.dump(row, f)
            f.write('\n')
    
    # remove output dir
    !rm -r "$OUTPUT_DIR"
    
    # do predict
    bert_predict()
    
    # read predictions
    with open(f'{PREDICTION_SAVE_PATH}/{PREDICTION_FILE}') as f:
        this_preds = f.read().split('\n')[:-1]
        bert_outputs += [pred.split() for pred in this_preds]

### Restore Dataset labels from predictions

In [ ]:
# get test sentences
test_sentences = [row['tokens'] for row in test_rows]

In [ ]:
bert_dataset_labels = [] # store all dataset labels for each publication

for length in paper_length:
    labels = set()
    for sentence, pred in zip(test_sentences[:length], bert_outputs[:length]):
        curr_phrase = ''
        for word, tag in zip(sentence, pred):
            if tag == 'B': # start a new phrase
                if curr_phrase:
                    labels.add(curr_phrase)
                    curr_phrase = ''
                curr_phrase = word
            elif tag == 'I' and curr_phrase: # continue the phrase
                curr_phrase += ' ' + word
            else: # end last phrase (if any)
                if curr_phrase:
                    labels.add(curr_phrase)
                    curr_phrase = ''
        # check if the label is the suffix of the sentence
        if curr_phrase:
            labels.add(curr_phrase)
            curr_phrase = ''
    
    # record dataset labels for this publication
    bert_dataset_labels.append(labels)

In [ ]:
# bert_dataset_labels

### Filter based on Jaccard score and clean

In [ ]:
def jaccard_similarity(s1, s2):
    l1 = s1.split(" ")
    l2 = s2.split(" ")    
    intersection = len(list(set(l1).intersection(l2)))
    union = (len(l1) + len(l2)) - intersection
    return float(intersection) / union

filtered_bert_labels = []

for labels in bert_dataset_labels:
    filtered = []
    
    for label in sorted(labels, key=len):
        label = clean_text(label)
        if len(filtered) == 0 or all(jaccard_similarity(label, got_label) < 0.75 for got_label in filtered):
            filtered.append(label)
    
    filtered_bert_labels.append('|'.join(filtered))

In [ ]:
# filtered_bert_labels
# final_predictions

# Aggregate final predictions and write submission file

In [ ]:
#TODO Score implementeren om weer te geven wat goed werkt. coledrige uses FBeta score. Performance, Confusion matrix could be useful. 
#TODO Find a way to check whether we are right. Can be done using the calculation of TP, TN, FP and FN.

In [ ]:
# CURRENT PROBLEM
# BERT does not predict every time, but Literal matching does, so all predictions are from literal matching and thus BERT prediction is not used

final_predictions = []
for literal_match, bert_pred in zip(literal_preds, filtered_bert_labels):
    if literal_match:
        final_predictions.append(literal_match)
    else:
        final_predictions.append(bert_pred)
        

In [ ]:
sample_submission = test[:PREDICTION_SIZE]

sample_submission['PredictionString'] = final_predictions
sample_submission['LiteralPrediction'] = literal_preds
sample_submission['BertPrediction'] = filtered_bert_labels
sample_submission['adni'] = filtered_bert_labels[1] * PREDICTION_SIZE
sample_submission

# filtered_bert_labels

In [ ]:
#TODO Score implementeren om weer te geven wat goed werkt. coledrige uses FBeta score. Performance, Confusion matrix could be useful. 
#TODO Find a way to check whether we are right. Can be done using the calculation of TP, TN, FP and FN.
# TP = 0
# FP = 0
# TN = 0
# FN = 0

def check_match(predictions,cleaned_label):
    for pred in predictions:
#         result = 0
#         print(f"predict:{totally_clean_text(pred)} \tcorrect:{cleaned_label}")
        t = totally_clean_text(pred)
        if t and t in cleaned_label:
            return True
    return False
#     return result >= min(len(predictions)-5,1)
        
def calc_accuracy(row_name, sample_submission):
    preds = [check_match(row[row_name].split('|'), row["cleaned_label"]) for k,row in sample_submission.iterrows()]

    return len(list(filter(lambda x: x,preds))) / PREDICTION_SIZE
    
acc_bert = calc_accuracy("BertPrediction", sample_submission)
acc_literal = calc_accuracy("LiteralPrediction", sample_submission)
acc_combined = calc_accuracy("PredictionString", sample_submission)
adni = calc_accuracy("adni", sample_submission)

print(acc_bert,acc_literal,acc_combined,adni)
# for k, row in sample_submission.iterrows():
#     id = row['Id']
#     predictions = row['PredictionString'].split('|')
# #     print(predictions)
    
#     for pred in predictions:
#         if totally_clean_text(pred) in sample_submission["cleaned_label"]:
#             row["Match"] = True
#         else:
#             row["Match"] = False
        


In [ ]:
sample_submission.to_csv(f'submission.csv', index=False)